In this notebook, I used the Planet-NICFI images that I previously exported and visually reviewed to check if the ASM site is still visible to create boxes and use them to export Sentinel-1 images. The index string of the Sentinel-1 image corresponds to the Planet-NICFI index, this way the images can be overlapped.

# 0) Import packages


In [ ]:
import os
import sys

import ee
import geemap
import json

from google.colab import files

In [ ]:
# clone repository to execute Sentinel-1 data preprocessing
!git clone https://github.com/adugnag/gee_s1_ard.git

sys.path.append('/content/gee_s1_ard/python-api')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Map = geemap.Map() # initialize GEE and geemap

In [ ]:
# import function for preprocessing
from wrapper import s1_preproc

# 1) Upload and visualize administrative borders

In [ ]:
# upload the geojson with the administrative borders (from GADM)
uploaded = files.upload()

In [ ]:
# get the filename
filename = next(iter(uploaded))

In [ ]:
# load the GeoJSON file into dictionary
with open(filename, 'r') as f:
    geojson_dict = json.load(f)

In [ ]:
# convert geojson to ee.FeatureCollection
ee_object = geemap.geojson_to_ee(geojson_dict)

In [ ]:
Map.addLayer(ee_object, {}, 'Area of interest')
Map.centerObject(ee_object)
Map

# 2) Access and preprocess Sentinel-1 data

In [ ]:
params = {
    'APPLY_BORDER_NOISE_CORRECTION': True,
    'APPLY_TERRAIN_FLATTENING': True,
    'APPLY_SPECKLE_FILTERING': False,
    'POLARIZATION': 'VVVH',
    'ORBIT': 'DESCENDING',
    'START_DATE': '2023-01-01',
    'STOP_DATE': '2023-06-30',
    'ROI': ee_object.geometry(),
    'PLATFORM_NUMBER': 'A',
    'ORBIT_NUM': None,
    'SPECKLE_FILTER_FRAMEWORK': 'MULTI',
    'SPECKLE_FILTER': 'GAMMA MAP',
    'SPECKLE_FILTER_KERNEL_SIZE': 3,
    'SPECKLE_FILTER_NR_OF_IMAGES': 10,
    'DEM': ee.Image('USGS/SRTMGL1_003'),
    'TERRAIN_FLATTENING_MODEL': 'VOLUME',
    'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER': 0,
    'FORMAT': 'DB',
    'CLIP_TO_ROI': False,
    'SAVE_ASSET': False,
    'ASSET_ID': None
}

In [ ]:
processed_s1_collection = s1_preproc(params)

In [ ]:
# select VV and VH band and make a median composite
s1_images_preprocessed = processed_s1_collection.select(['VV', 'VH']).mean()

In [ ]:
Map.addLayer(s1_images_preprocessed, {'min': -25, 'max': 5, 'bands': 'VV'}, f'Sentinel-1 VV')
Map.addLayer(s1_images_preprocessed, {'min': -25, 'max': 5, 'bands': 'VH'}, f'Sentinel-1 VH')
Map

# 3) Make grid and export tiles

In [ ]:
def generate_grid(region, scale, pixelWidth=375, pixelHeight=375):
    bounds = region.geometry().bounds()
    coords = ee.List(bounds.coordinates().get(0))

    # extract the coordinates from AOI
    xmin = ee.List(coords.get(0)).get(0)
    ymin = ee.List(coords.get(0)).get(1)
    xmax = ee.List(coords.get(2)).get(0)
    ymax = ee.List(coords.get(2)).get(1)

    # calculate the width and height in meters of each tile
    width_in_meters = ee.Number(pixelWidth).multiply(scale)
    height_in_meters = ee.Number(pixelHeight).multiply(scale)

    # convert meters to degrees approximately for longitude and latitude
    # this is done in an approximate way, using a conversion factor for degrees
    dx = width_in_meters.divide(111320)
    dy = height_in_meters.divide(110540)

    # create sequences for longitude and latitude to generate grid points
    longs = ee.List.sequence(xmin, xmax, dx)
    lats = ee.List.sequence(ymax, ymin, dy.multiply(-1))  # ensure decrement for latitude

    # helper function to create grid rectangles
    def make_rects_lon(lon):
        lon = ee.Number(lon)  # lon must be ee.Number for arithmetic operations
        def make_rects_lat(lat):
            lat = ee.Number(lat)  # same for lat
            rect = ee.Geometry.Rectangle([lon, lat, lon.add(dx), lat.add(dy)])
            return ee.Feature(rect)

        return lats.map(make_rects_lat)

    # make the grid and flatten the resulting list of lists
    rects = longs.map(make_rects_lon).flatten()
    grid = ee.FeatureCollection(rects)

    return grid

def save_progress(index):
    with open("export_progress.txt", "w") as file:
        file.write(str(index))

def load_progress():
    try:
        with open("export_progress.txt", "r") as file:
            return int(file.read())
    except FileNotFoundError:
        return 0  # default to 0 if the progress file doesn't exist

def export_single_tile(image, tile, folder_name, index, scale=10):
    clipped_image = image.clip(tile.geometry())

    # include index in the image name
    description = f's1_{index}'

    task = ee.batch.Export.image.toDrive(
        image=clipped_image,
        description=description,
        folder=folder_name,
        scale=scale,
        maxPixels=1e13
    )
    task.start()

def export_tiles(image_collection, grid, aoi, folder_name, scale=10):
    # load the last completed tile index
    start_index = load_progress()
    total_tiles = grid.size().getInfo()

    # filter the grid to include only cells that intersect the AOI
    intersecting_cells = grid.filterBounds(aoi)

    # get list of intersecting cells
    intersecting_list = intersecting_cells.toList(intersecting_cells.size())

    # update the total number of tiles to the number of intersecting cells
    total_tiles = intersecting_cells.size().getInfo()

    for i in range(start_index, total_tiles):
        # get the current tile
        tile = ee.Feature(intersecting_list.get(i))
        description = f's1_{i}'

        # start the export task
        task = ee.batch.Export.image.toDrive(
            image=image_collection,
            description=description,
            folder=folder_name,
            scale=scale,
            region=tile.geometry().bounds(),  # use the bounds of the tile
            maxPixels=1e13
        )
        task.start()

        # save progress after each successful submission
        save_progress(i + 1)
        print(f'Submitted tile {i+1} of {total_tiles}')

In [ ]:
# generate grid
grid_tiles = generate_grid(ee_object, 10, 180, 180)

In [ ]:
# run the export
# when the process stops because of too many tasks submitted, after a while just
# run again this line; it will read the export_progress file and start over from
# where it stopped
export_tiles(s1_images_preprocessed, grid_tiles, ee_object, "s1_tiles", 10)

In [ ]:
# !earthengine task cancel all